# Purpose

This notebook is setup to query the [OSTI.gov](https://www.OSTI.gov) API for project records. The goals for the code located herein are:

1. Determine what fields are available for different records in OSTI
2. Design a DOE Solar Energy Technologies Office (SETO) query that only pulls that technology office's data
    * **Note:** this required `{'sponsoring_org': '"EE-4S"'}` in order to work the same as the browser-based search query. It appears that the syntax of the API and of the browser-based search is not fully harmonized right now, although I'm told it will be in the future.
3. Build the query to work using an arbitrarily-large list of formatted project IDs, assuming the Solar Information Management System (SIMS) project code syntax as the input.
    * **Note:** SIMS is an internal DOE system


# The Main Event

If all you want to do is extract records from OSTI's API, you needn't go any further than the next few cells. Anything past this section is simply testing code for trying to improve our queries, but isn't needed for the most basic functionality.

In [1]:
#First things first, make sure we can import modifications to custom packages
import autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt
import pandas as pd
import math
import os
import osti

In [3]:
#First, need to run a single query to get the full results count
params = {'sort': 'publication_date desc', 'sponsor_org': '"EE-4S"', 'page': 1}

_, results_count = osti.query_API(params = params, print_status = True)

Query was successful: True

Query made on Fri, 01 Mar 2019 14:08:33 GMT returned 2408 hits

URL used was https://www.osti.gov/api/v1/records?sort=publication_date+desc&sponsor_org=%22EE-4S%22&page=1&publication_date_start=01%2F01%2F1980&publication_date_end=03%2F01%2F2019


## Make sure you change start and end dates here if you need to

In [4]:
# Pull the full results of your query into a DataFrame

# format for dates = 'MM/DD/YYYY'
query_start_date = '01/01/1980'
query_end_date = dt.date.today().strftime('%m/%d/%Y')  # today's date

params = {'sort': 'publication_date desc', 'sponsor_org': '"EE-4S"',
          'rows': results_count}

temp, _ = osti.query_API(params=params)
df = pd.DataFrame.from_dict(temp)

In [5]:
#Extract the link for the landing page of each record
df['citation_link'] = df['links'].apply(osti.get_citation_URL)

In [6]:
#Export data to a CSV file, named using today's date
today = dt.date.today().strftime('%m-%d-%Y')
filename = today + '_OSTI_Data_Pull.csv'

df.to_csv(os.path.join(r'output/',filename))

# Cleaning and Exploring the Data

## Checking Consistency of Office Specificity

As multiple DOE program offices can be associated with a project output in the OSTI database, I want to check to make sure that the filter we're applying to get only solar-related projects is working as expected.

In [ ]:
#Convert sponsoring_org field to be a str instead of list and split on the comma delimiter to make sure 
    #EE-4S is everywhere
df['sponsor_orgs'].astype('str').str.split(", ", expand = True)[1].value_counts()

## Determining What Links We Get and Where They Go

The `links` field seems to provide some URLs for us to use, let's do some spot checks and see if they go to project landing pages or straight to the full text itself (the former is preferred over the latter).

**It looks like `'rel': 'citation'` is the link type we want to go straight to the landing page.** `'href'` key for dict gives us what we're looking for.

In [ ]:
df['citation_link'] = df['links'].apply(osti.get_citation_URL)

## What Does an Embargoed Document Look Like?

We know that certain projects can embargo their publications for different periods of time (and some, such as the SBIR program, may even *require* embargoes). As such, we want to be able to flag a record when we think it's under embargo so we can check back regularly to see when the document may finally be available to the public. So, we need to figure out if there's a pattern that we can use to generate such a flag.

**Let's first look for award EE0007326 and OSTI report ID 1490198, which I know to be currently under embargo.**

In [ ]:
params = {'sort': 'publication_date desc', 'sponsor_org': '"EE-4S"',
         'identifier': 'EE0007326'}

r = requests.get(URL, params=params)

query_date = r.headers["Date"]
results_count = r.headers['X-Total-Count']

print(f"Query was successful: {r.status_code == requests.codes.ok}")
print(f"Query made on {query_date} returned {results_count} hits")
print(f"URL used was {r.url}")


In [ ]:
df_test = pd.DataFrame.from_dict(r.json())
df_test

**Sadly, as of right now, it appears there's no good way to determine if a record is embargoed.** I thought the key may have been in the `links` column (those with only a citation-type link could be considered embargoed), but the ones lacking a full text may simply be behind a paywall of a journal, for example. So right now, I see no fool-proof way to flag these.